<a href="https://colab.research.google.com/github/angeruzzi/MachineLearningRef/blob/main/Preprocessing_Categorical_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bibliotecas e Fontes

In [ ]:
!pip install category_encoders 

In [ ]:
#Bibliotecas Base
import pandas as pd
import numpy as np
from math import sqrt

from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#Base de Dados
fonte_treino = 'https://raw.githubusercontent.com/angeruzzi/Datasource/main/DesafioDDS202205_treino.csv'
fonte_teste = 'https://raw.githubusercontent.com/angeruzzi/Datasource/main/DesafioDDS202205_teste.csv'
orig   = pd.read_csv(fonte_treino, sep = ',', decimal = '.')
teste  = pd.read_csv(fonte_teste, sep = ',', decimal = '.')

#Tratamentos

In [ ]:
treino = orig.copy()
X_train = orig.drop(['alugueis'], axis = 1) 
y_train = orig['alugueis']

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

##One Hot Encoder - 1

In [ ]:
enc = OneHotEncoder(cols=['dia', 'estacao', 'feriado'], use_cat_names=True)
enc.fit(X_train)

X_train1 = enc.transform(X_train)
X_test1 = enc.transform(X_test)

X_train1.head()

,hora,dia_sabado,dia_terca,dia_segunda,dia_sexta,dia_quinta,dia_domingo,dia_quarta,feriado_nao,feriado_sim,estacao_outono,estacao_inverno,estacao_primavera,estacao_verao,temperatura,chuva,umidade,sol,visibilidade,vento
2740,11,1,0,0,0,0,0,0,1,0,1,0,0,0,25.3,0.0,49.8,2.01,97.0,5.17
2420,9,0,1,0,0,0,0,0,1,0,0,1,0,0,17.4,0.0,60.3,0.54,35.0,1.93
2292,0,1,0,0,0,0,0,0,1,0,0,0,1,0,15.5,0.0,58.1,0.00,65.0,3.37
2656,13,0,0,1,0,0,0,0,1,0,1,0,0,0,28.9,0.0,45.2,2.59,92.0,2.05
1411,7,0,0,1,0,0,0,0,1,0,1,0,0,0,31.7,0.0,91.2,0.03,22.0,2.53


##Ordinal - 2

In [ ]:
enc2 = OrdinalEncoder(cols=['dia', 'estacao', 'feriado'])
enc2.fit(X_train)

X_train2 = enc2.transform(X_train)
X_test2 = enc2.transform(X_test)

X_train2.head()

,hora,dia,feriado,estacao,temperatura,chuva,umidade,sol,visibilidade,vento
2740,11,1,1,1,25.3,0.0,49.8,2.01,97.0,5.17
2420,9,2,1,2,17.4,0.0,60.3,0.54,35.0,1.93
2292,0,1,1,3,15.5,0.0,58.1,0.00,65.0,3.37
2656,13,3,1,1,28.9,0.0,45.2,2.59,92.0,2.05
1411,7,3,1,1,31.7,0.0,91.2,0.03,22.0,2.53


##Target Encoding - 3

In [ ]:
enc3 = TargetEncoder(cols=['dia', 'estacao', 'feriado'])
enc3.fit(X_train, y_train)

X_train3 = enc3.transform(X_train)
X_test3 = enc3.transform(X_test)

X_train3.head()

/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:94: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:99: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


,hora,dia,feriado,estacao,temperatura,chuva,umidade,sol,visibilidade,vento
2740,11,765.748837,737.490507,858.554245,25.3,0.0,49.8,2.01,97.0,5.17
2420,9,681.223124,737.490507,237.741573,17.4,0.0,60.3,0.54,35.0,1.93
2292,0,765.748837,737.490507,741.512644,15.5,0.0,58.1,0.00,65.0,3.37
2656,13,728.700624,737.490507,858.554245,28.9,0.0,45.2,2.59,92.0,2.05
1411,7,728.700624,737.490507,858.554245,31.7,0.0,91.2,0.03,22.0,2.53


#Modelagem

##Random Forest

In [ ]:
mdl1 = RandomForestRegressor(n_jobs=6, n_estimators=100, random_state=42)
mdl1.fit(X_train1, y_train)

predict_1 = mdl1.predict(X_test1)

mae1 = mean_absolute_error(y_test, predict_1)
mse1 = mean_squared_error(y_test, predict_1)
rmse1 = sqrt(mse1)

print(mae1)
print(rmse1)

176.95467555555555
298.7438221955393


In [ ]:
mdl2 = RandomForestRegressor(n_jobs=6, n_estimators=100, random_state=42)
mdl2.fit(X_train2, y_train)

predict_2 = mdl2.predict(X_test2)

mae2 = mean_absolute_error(y_test, predict_2)
mse2 = mean_squared_error(y_test, predict_2)
rmse2 = sqrt(mse2)

print(mae2)
print(rmse2)

180.09087111111108
303.88312908060925


In [ ]:
mdl3 = RandomForestRegressor(n_jobs=6, n_estimators=100, random_state=42)
mdl3.fit(X_train3, y_train)

predict_3 = mdl2.predict(X_test3)

mae3 = mean_absolute_error(y_test, predict_3)
mse3 = mean_squared_error(y_test, predict_3)
rmse3 = sqrt(mse3)

print(mae3)
print(rmse3)

214.37591999999998
342.99013793570737
